In [2]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from dash import Dash, html, dash_table, dcc, Output, Input
import dash_bootstrap_components as dbc

Import data

In [3]:
pp_db_raw = pd.read_csv("global_power_plant_database.csv", dtype={'other_fuel3': str})
cont = pd.read_csv("list-of-countries-by-continent-2024.csv")

Check columns and merge on country_long and country column

In [4]:

print("Before merge", pp_db_raw.shape)
print("Power Plant DB columns: ", pp_db_raw.columns)
print("Country & Continent CSV file columns: ", cont.columns)
pp_db = pp_db_raw.merge(cont, how='left', left_on='country_long', right_on='country', suffixes=('_short', ''))
print("After merge", pp_db.shape) # 3 columns added and same number of rows due to left join


Before merge (34936, 36)
Power Plant DB columns:  Index(['country', 'country_long', 'name', 'gppd_idnr', 'capacity_mw',
       'latitude', 'longitude', 'primary_fuel', 'other_fuel1', 'other_fuel2',
       'other_fuel3', 'commissioning_year', 'owner', 'source', 'url',
       'geolocation_source', 'wepp_id', 'year_of_capacity_data',
       'generation_gwh_2013', 'generation_gwh_2014', 'generation_gwh_2015',
       'generation_gwh_2016', 'generation_gwh_2017', 'generation_gwh_2018',
       'generation_gwh_2019', 'generation_data_source',
       'estimated_generation_gwh_2013', 'estimated_generation_gwh_2014',
       'estimated_generation_gwh_2015', 'estimated_generation_gwh_2016',
       'estimated_generation_gwh_2017', 'estimated_generation_note_2013',
       'estimated_generation_note_2014', 'estimated_generation_note_2015',
       'estimated_generation_note_2016', 'estimated_generation_note_2017'],
      dtype='object')
Country & Continent CSV file columns:  Index(['country', 'continen

In [5]:
pp_db.describe()

,capacity_mw,latitude,longitude,commissioning_year,year_of_capacity_data,generation_gwh_2013,generation_gwh_2014,generation_gwh_2015,generation_gwh_2016,generation_gwh_2017,generation_gwh_2018,generation_gwh_2019,estimated_generation_gwh_2013,estimated_generation_gwh_2014,estimated_generation_gwh_2015,estimated_generation_gwh_2016,estimated_generation_gwh_2017
count,34936.000000,34936.000000,34936.000000,17447.000000,14887.000000,6417.000000,7226.000000,8203.000000,9144.000000,9500.000000,9637.000000,9659.000000,16120.000000,16503.000000,17050.000000,17570.000000,33138.000000
mean,163.355148,32.816637,-6.972803,1997.414823,2018.218849,592.696107,656.863891,762.368840,693.149763,661.834161,517.320785,423.922436,239.112221,242.431122,235.866067,235.695265,716.443378
std,489.636072,22.638603,78.405850,23.397835,1.606428,2174.833482,2231.464288,2533.459828,2404.516759,2369.296957,1975.302686,1698.506485,1191.258172,1285.798297,1267.005223,1259.559841,2484.361200
min,1.000000,-77.847000,-179.977700,1896.000000,2000.000000,-947.600000,-989.619000,-864.428000,-768.620000,-934.944000,-982.622000,-780.339000,1.120000,0.870000,0.440000,0.300000,0.000000
25%,4.900000,29.256475,-77.641550,1988.000000,2017.000000,1.947000,2.260000,2.664500,2.729000,2.467000,2.242000,2.751500,8.620000,8.680000,8.380000,8.322500,8.180000
50%,16.745000,39.727750,-2.127100,2007.000000,2019.000000,23.426000,23.608500,26.142000,22.462389,17.876500,12.530000,11.530000,27.620000,28.250000,26.830000,27.555000,37.590000
75%,75.344250,46.263125,49.502675,2014.000000,2019.000000,199.706000,226.322250,285.862583,249.866750,214.505150,151.117000,122.781498,106.805000,106.980000,103.120000,107.237500,229.565000
max,22500.000000,71.292000,179.388700,2020.000000,2019.000000,50834.000000,32320.917000,37433.607000,32377.477000,36448.643000,35136.000000,31920.368000,48675.060000,58470.770000,57113.350000,60859.730000,82810.770000


Check records missing country field to see if merge was successful

In [6]:

print(pp_db[['country_short', 'country_long', 'country', 'continent']].isna().sum())
# country column in merged table has 73 missing values, we check which records these correspond to in raw table
print(pp_db_raw[pp_db['country'].isna()]['country_long'].unique())
# Note that 2 power plants are located in Antarctica which we add to Oceania (New Zealand)
print("Antarctica Power Plants: ", pp_db.loc[pp_db['country_long']=='Antarctica', :'primary_fuel'])

country_short     0
country_long      0
country          73
continent        73
dtype: int64
['Antarctica' 'Brunei Darussalam' 'Congo' 'Cote DIvoire'
 'Democratic Republic of the Congo' 'Macedonia' 'Swaziland'
 'Syrian Arab Republic']
Antarctica Power Plants:     country_short country_long                       name   gppd_idnr  \
90           ATA   Antarctica  McMurdo Station Generator  WRI1023843   
91           ATA   Antarctica                Ross Island  WRI1022458   

    capacity_mw  latitude  longitude primary_fuel  
90          6.6  -77.8470   166.6605          Oil  
91          1.0  -77.8428   166.7271         Wind  



Comparing output to list-of-countries-by-continent-2024.csv , we make the following changes manually:

Swaziland --> Eswatini, Brunei Darussalam --> Brunei

Congo --> Republic of the Congo, Cote DIvoire --> Ivory Coast

Democratic Republic of the Congo --> DR Congo, Macedonia --> North Macedonia

Syrian Arab Republic --> Syria

We also  make these changes to the raw data in the continent CSV file:

Add Kosovo, United States --> United States of America

Antarctica --> New Zealand


In [7]:
replacement_dict={
    'Swaziland': 'Eswatini',
    'Brunei Darussalam': 'Brunei',
    'Congo': 'Republic of the Congo',
    'Cote DIvoire': 'Ivory Coast',
    'Democratic Republic of the Congo': 'DR Congo',
    'Macedonia': 'North Macedonia',
    'Syrian Arab Republic': 'Syria',
    'Antarctica': 'New Zealand'
}
pp_db_raw['country_long'] = pp_db_raw['country_long'].replace(replacement_dict)

pp_db = pp_db_raw.merge(cont, how='left', left_on='country_long', right_on='country', suffixes=('_short', ''))

print(pp_db[['country_short', 'country_long', 'country', 'continent']].isna().sum())

country_short    0
country_long     0
country          0
continent        0
dtype: int64


Creating new columns for renewable vs non-renewable energy and 'Other' primary fuel_type by continent and world categories for data visualisation. Aim is that any primary fuel type that represents less than 1% of the output for a continent will be added to an 'Other' category.

In [8]:
print(pp_db['primary_fuel'].unique())
renewable_energies = ['Hydro', 'Solar', 'Wind', 'Biomass', 'Wave and Tidal', 'Geothermal']
non_renewable_energies = ['Nuclear', 'Coal', 'Gas', 'Oil', 'Petcoke',]
other = ['Other', 'Waste', 'Cogeneration', 'Storage']

conditions = [
    pp_db['primary_fuel'].isin(renewable_energies),
    pp_db['primary_fuel'].isin(non_renewable_energies),
    pp_db['primary_fuel'].isin(other)
]

# Define the categories
choices = ['Renewable', 'Non-Renewable', 'Other']

# Create new column
pp_db['renew_energy'] = np.select(conditions, choices, default='Unknown')

['Hydro' 'Solar' 'Gas' 'Other' 'Oil' 'Wind' 'Nuclear' 'Coal' 'Waste'
 'Biomass' 'Wave and Tidal' 'Petcoke' 'Geothermal' 'Storage'
 'Cogeneration']


In [53]:
# Creating new column called text to use as data that shows ehn we hover over points in the world map figure
pp_db['text'] = pp_db['name'] + ', ' + pp_db['country']

###### Create web app with Plotly/Dash #######

app = Dash(external_stylesheets=[dbc.themes.SLATE])

fuel_color_map = {
    # Renewable Energy Sources
    'Hydro': '#2ECC71',        # Light Green
    'Solar': '#F1C40F',        # Yellow
    'Wind': '#3498DB',         # Light Blue
    'Biomass': '#27AE60',      # Green
    'Wave and Tidal': '#D35400',  # Teal
    'Geothermal': '#9B59B6',   # Purple

    # Non-Renewable Energy Sources
    'Nuclear': '#34495E',      # Dark Blue-Grey
    'Coal': '#E74C3C',         # Red
    'Gas': '#7F8C8D',          # Dark Orange
    'Oil': '#1ABC9C',          # Darker Orange
    'Petcoke': '#95A5A6',      # Grey

    # Other Categories
    'Other': '#BDC3C7',        # Light Grey
    'Waste': '#E67E22',        # Dark Grey
    'Cogeneration': '#16A085', # Dark Teal
    'Storage': '#2980B9',      # Blue

    # Category Labels (Overall)
    'Renewable': '#2ECC71',    # Light Green
    'Non-Renewable': '#E74C3C', # Red
}

custom_colors = [
    "#FF5733",  # Red-Orange
    "#33FF57",  # Green
    "#3357FF",  # Blue
    "#F39C12",  # Orange
    "#9B59B6",  # Purple
    "#E74C3C",  # Red
    "#1ABC9C",  # Teal
    "#8E44AD",  # Dark Purple
    "#3498DB",  # Light Blue
    "#2ECC71",  # Light Green
    "#E67E22",  # Dark Orange
    "#ECF0F1",  # Light Grey
    "#95A5A6",  # Grey
    "#34495E",  # Dark Blue-Grey
    "#16A085"   # Dark Teal
]

# Creating the map
map_columns=['country_long', 'name', 'capacity_mw', 'primary_fuel', 'longitude', 'latitude', 'text']
map_df = pp_db.sort_values(by='capacity_mw', ascending=False).head(500)[map_columns]
map_fig = px.scatter_geo(
    data_frame=map_df,
    lat='latitude',
    lon='longitude',
    color='primary_fuel',
    hover_name='text',
    color_discrete_sequence=custom_colors,
    size='capacity_mw',
    opacity=0.7,
    projection='natural earth',
)

# Update marker border properties
map_fig.update_traces(
    marker=dict(
        line=dict(
            color='black',  # Border color
            width=0.1  # Border width
        )
    )
)

# Changing map appearance
map_fig.update_layout(
    template='plotly_dark',
    plot_bgcolor= 'rgba(0, 0, 0, 0)',
    paper_bgcolor= 'rgba(0, 0, 0, 0)',
    font=dict(color='white'),  # Font color
    geo=dict(
        showland=True, landcolor='#333333',  # Dark land color
        showocean=True, oceancolor='#444444',  # Dark ocean color
        lakecolor='#444444',  # Dark lake color
    ),
    title_font=dict(color='white'),  # Title font color
)

# Creating functions for different card bodies in the dashboard (runs faster and is less messy)
def drawTitle():
    return html.Div([
        dbc.Card(
            dbc.CardBody([
                html.Div([
                    html.H1("Dashboard for EDA of Global Power Plants Database"),
                        ], style={'textAlign': 'center',  'color': 'white'}) 
            ])
        ),
    ])

def drawDropdown():
    return html.Div([
        dbc.Card(
            dbc.CardBody([
                html.Div([
                    html.H3("Select Continent:"),
                ], style={'textAlign': 'center',  'color': 'white'}),
                dcc.Dropdown(id='cont_dropdown',
                        options = [{'label': continent, 'value': continent} for continent in pp_db['continent'].unique()] + [{'label': 'World', 'value': 'All'}],
                        value='Europe'), 
            ])
        ),
    ])

def drawTable():
    return html.Div([
        dbc.Card(
            dbc.CardBody([
                html.H5("Power Plants Table Sorted by Capacity", style={'textAlign': 'center', 'color': 'white'}),
                html.Br(),
                dash_table.DataTable(
                    id='power-plants-table',
                    columns=[
                        {'name': 'Country', 'id': 'country_long'},
                        {'name': 'Name', 'id': 'name'},
                        {'name': 'Capacity (MW)', 'id': 'capacity_mw'},
                        {'name': 'Primary Fuel', 'id': 'primary_fuel'}
                    ],
                    data=[],
                    page_size=13,
                    style_header={
                        'backgroundColor': '#1f1f1f',  # Dark background for header
                        'color': 'white',              # White text for header
                        'border': '1px solid #444'     # Border color for header
                    },
                    style_cell={
                        'backgroundColor': '#333333',  # Dark background for cells
                        'color': 'white',              # White text for cells
                        'textAlign': 'left',           # Align text to the left
                        'border': '1px solid #444'     # Border color for cells
                    },
                    style_data_conditional=[
                        {
                            'if': {'row_index': 'odd'},  # Alternate row styling for better readability
                            'backgroundColor': '#2d2d2d'  # Slightly lighter dark background for odd rows
                        }
                    ],
                    fill_width=True
                )
            ])
        ),
    ])

def drawPieChart():
    return  html.Div([
        dbc.Card(
            dbc.CardBody([html.H5('Proportion of Total Power Plant Capacity', style={'textAlign': 'center', 'color': 'white'}),
                          dcc.RadioItems(id='pie_radioitems',
                        options=[{'label':' Primary Fuel', 'value':'primary_fuel'},
                                {'label':' Renewables', 'value':'renew_energy'}],
                        value='primary_fuel',
                        inline=True),
                        dcc.Graph(id='pie_chart_cap'),
            ])
        ),  
    ])

def drawMap():
    return  html.Div([
        dbc.Card(
            dbc.CardBody([html.H5("Distribution of 6000 Largest Power Plants by Capacity", style={'textAlign': 'center',  'color': 'white'}),
                         dcc.Graph(figure=map_fig)
            ])
        ),  
    ])

def drawHist():
    return  html.Div([
        dbc.Card(
            dbc.CardBody([
                        dcc.Graph(id='capacity_hist')
            ])
        ),  
    ])

 # Actual app layout
app.layout = html.Div([
    dbc.Card(
        dbc.CardBody([
            dbc.Row([
                dbc.Col(drawTitle(), width=12),
            ], align='center'),
            dbc.Row([
                dbc.Col(drawDropdown(), width=4),
            ], justify='center', align='center'),
            dbc.Row([
                dbc.Col([
                    drawHist() 
                ], width=6),
                dbc.Col([
                    drawMap()
                ], width=6),
            ], align='center'), 
            dbc.Row([
                dbc.Col([
                    drawPieChart()
                ], width=6),
                dbc.Col([
                    drawTable()
                ], width=6),
            ], align='center'),      
        ]), color = 'dark'
    )
], style={'height': '100vh', 'overflow': 'auto'})

# Callback function for updating histogram
@app.callback(Output('capacity_hist', 'figure'),
              Input('cont_dropdown', 'value'))
def sync_input_hist(cont_selection):
    if cont_selection!='All':
        continent_filtered = pp_db.loc[pp_db['continent'] == cont_selection].copy()
    else:
        continent_filtered = pp_db
    continent_filtered['log_capacity_mw'] = np.log10(continent_filtered.loc[:,'capacity_mw'])
    fig = px.histogram(continent_filtered,  
                       x='log_capacity_mw',
                       hover_name='text',
                       color='primary_fuel',
                       log_y=True,
                       nbins=100)
    
    tick_values = np.log10(np.array([1, 10, 100, 500, 1000, 5000, 10000]))
    tick_texts = ['1', '10', '100', '500', '1K', '5K', '10K']

    fig.update_layout(
        template='plotly_dark',
        plot_bgcolor= 'rgba(0, 0, 0, 0)',
        paper_bgcolor= 'rgba(0, 0, 0, 0)',   # Background color for the plot area
        font=dict(color='white'),  # Font color for axes and labels
        title_font=dict(color='white'),  # Title font color
        xaxis=dict(
            title='Capacity (MW)', showgrid=True,
            gridcolor='rgba(255, 255, 255, 0.1)', color='white',
            tickmode='array', tickvals=tick_values, ticktext=tick_texts,
            range=[0, continent_filtered['log_capacity_mw'].max()+1]  # This sets the range from 10^0 to 10^5
                            ),
        yaxis=dict(
            title='Count', showgrid=True, gridcolor='rgba(255, 255, 255, 0.1)', color='white'
        ),
        bargap=0.1
    )

    fig.update_layout(barmode='overlay')
    return fig


# Callback function for updating Pie chart

@app.callback(Output('pie_chart_cap', 'figure'),
              Input('cont_dropdown', 'value'),
              Input('pie_radioitems', 'value'))
def sync_input_pie_chart_cap(cont_selection, fuel_or_renew):
    if cont_selection!='All':
        continent_filtered = pp_db.loc[pp_db['continent'] == cont_selection].copy()
    else:
        continent_filtered = pp_db
    grouped_by_fuel = continent_filtered.groupby(fuel_or_renew)['capacity_mw'].sum().reset_index()
    fig = px.pie(grouped_by_fuel, 
                 values='capacity_mw',
                 names=fuel_or_renew,
                 color=fuel_or_renew,
                 color_discrete_map=fuel_color_map)
    fig.update_layout(
        template='plotly_dark',
        plot_bgcolor= 'rgba(0, 0, 0, 0)',
        paper_bgcolor= 'rgba(0, 0, 0, 0)',  # Background color for the entire plot
        font=dict(color='white'),  # Font color for pie chart labels
        title_font=dict(color='white'),  # Title font color
    )

    return fig

# Callback function for updating Table

@app.callback(
    Output('power-plants-table', 'data'),
    [Input('cont_dropdown', 'value')]
)
def update_table(cont_selection):
    if cont_selection != 'All':
        filtered_data = pp_db[pp_db['continent'] == cont_selection]
    else:
        filtered_data = pp_db
    
    sorted_data = filtered_data[['country_long', 'name', 'capacity_mw', 'primary_fuel']].sort_values('capacity_mw', ascending=False)

    return sorted_data.to_dict('records')

# Run the dashboard
app.run(jupyter_mode="external")
#app.run_server(debug=True, use_reloader=False)  # Turn off reloader if inside Jupyter


Dash app running on http://127.0.0.1:8050/


In [32]:
pp_db['capacity_mw'].describe()

count    34936.000000
mean       163.355148
std        489.636072
min          1.000000
25%          4.900000
50%         16.745000
75%         75.344250
max      22500.000000
Name: capacity_mw, dtype: float64